In [6]:
import os
import collections as col
import importlib
import operator
import math

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines

plot_aux_module = '/home/local/work/code/github/project-diploid-assembly/notebooks/aux_mods/plot_aux.py'
plot_aux_spec = importlib.util.spec_from_file_location("plot_aux", plot_aux_module)
plot_aux = importlib.util.module_from_spec(plot_aux_spec)
plot_aux_spec.loader.exec_module(plot_aux)

samples = plot_aux.load_sample_table()
hexcodes, rgbcodes, popmap = plot_aux.load_population_annotation()

prop = plot_aux.get_plot_property

# color_version
# - full
# - super_pop
# - red_black
color_version = 'red_black'

if color_version == 'full' or color_version == 'super_pop':
    rgbcodes['HiFi'] = tuple(map(lambda x: round(x/255, 2), (35, 35, 35)))
    rgbcodes['CLR'] = tuple(map(lambda x: round(x/255, 2), (35, 35, 35)))
elif color_version == 'red_black':
    rgbcodes['HiFi'] = tuple(map(lambda x: round(x/255, 2), (178, 34, 34)))
    rgbcodes['CLR'] = (0, 0, 0)
else:
    raise

save_plots = False
print_stats = True

pipeline_version = 'v12'

stats_path = '/home/local/work/data/hgsvc/fig1_panels/kmer_stats'
out_path = os.path.join(stats_path, pipeline_version, 'fig1_panel_kmer-stats')
cache_file = os.path.join(stats_path, 'cache_{}.h5'.format(pipeline_version))

keep_combinations = {
    '0': 'Illumina_only',
    '1': 'Illumina_only',
    '0-1': 'Illumina_only',
    '2': 'Hap1_only',
    '3': 'Hap2_only',
    '2-3': 'Hap_only',
    '0-2-3': 'Hap_Illumina',
    '1-2-3': 'Hap_Illumina',
    '0-1-2-3': 'Hap_Illumina',
    '0-2': 'Hap1_Illumina',
    '1-2': 'Hap1_Illumina',
    '0-1-2': 'Hap1_Illumina',
    '0-3': 'Hap2_Illumina',
    '1-3': 'Hap2_Illumina',
    '0-1-3': 'Hap2_Illumina',
    '2-4': 'Hap1_GRCh38',
    '3-4': 'Hap2_GRCh38',
    '4': 'GRCh38',
    '2-3-4': 'Hap_GRCh38',
    '0-4': 'GRCh38_Illumina',
    '1-4': 'GRCh38_Illumina',
    '0-1-4': 'GRCh38_Illumina',
}


def load_kmer_stats(file_path):
    kmer_stats = col.Counter()
    with open(file_path, 'r') as table:
        for line in table:
            if line.startswith('color'):
                continue
            _, combination, count = line.strip().split()
            if '5' in combination:
                # ignore all kmers that may be mitochondrial
                continue
            try:
                key = keep_combinations[combination]
            except KeyError:
                key = 'GRCh38_shared'
            kmer_stats[key] += int(count)
    return kmer_stats


if not os.path.isfile(cache_file):
    stat_files = plot_aux.load_plot_data_files(
        stats_path,
        '.kmer-counts.tsv',
        pipeline_version)
    plot_data = [load_kmer_stats(fp) for fp in stat_files]

    sample_platform = [
        plot_aux.extract_sample_platform(
            os.path.basename(f),
            multi_readset=True,
        ) for f in stat_files
    ]

    row_index = []
    for (sample, platform) in sample_platform:
        super_pop = samples[sample]['super_population']
        pop = samples[sample]['population']
        row_index.append((sample, super_pop, pop, platform, 'HAP', 0))

    df = pd.DataFrame(
        plot_data,
        columns=sorted(plot_data[0].keys()),
        index=row_index
    )

    df.index = pd.MultiIndex.from_tuples(
        df.index.values,
        names=['sample', 'super_pop', 'pop', 'platform', 'assembly', 'hap']
    )
    df.to_hdf(cache_file, key='cache', mode='w', format='fixed')
    
df = pd.read_hdf(cache_file, 'cache')
df.sort_index(axis=0, inplace=True, level=['sample', 'super_pop', 'pop', 'platform', 'assembly', 'hap'])        

errors, missing = plot_aux.check_cache_consistency(df)
if errors:
    raise ValueErrors('Skip sample(s) in cache: {}'.format(errors))
if missing:
    print(sorted(missing))
    print('---------------')
  
    
def plot_kmer_stats(sample_stats):
    
    fig, axis = plt.subplots(figsize=(8,8))
    
    xya_pairs = [
        ('Hap_Illumina', 'Hap_GRCh38', 0.33),
        ('Hap1_Illumina', 'Hap1_GRCh38', 1),
        ('Hap2_Illumina', 'Hap2_GRCh38', 1)
    ]

    for platform in plot_aux.get_sequencing_platforms():
        subset = sample_stats.xs(platform, level='platform', drop_level=False)
        for x, y, alpha in xya_pairs:
            x_values = subset[x].values.astype('float32')
            x_values /= 1e6
            y_values = subset[y].values.astype('float32')
            y_values /= 1e6
            
            if color_version == 'full':
                colors = [rgbcodes[i] for i in subset.index.get_level_values('pop')]
            elif color_version == 'super_pop':
                colors = [rgbcodes[i] for i in subset.index.get_level_values('super_pop')]
            elif color_version == 'red_black':
                colors = [rgbcodes[i] for i in subset.index.get_level_values('platform')]
            else:
                raise
            labels = [s + '-{}-{}'.format(platform, x.split('_')[0]) for s in subset.index.get_level_values('sample')]
            
            axis.scatter(
                x_values,
                y_values,
                s=prop('plot_marker_size'),
                c=colors,
                marker=prop('{}_marker'.format(platform)),
                label=labels,
                alpha=alpha
            )

    axis.set_xlabel(
        'Non-reference 31-mers\nw/ Illumina support (million)',
        fontsize=prop('fontsize_legend')
    )
    axis.set_ylabel(
        'Non-Illumina 31-mers\nw/ reference support (million)',
        fontsize=prop('fontsize_legend')
    )
    
    axis.tick_params(
        axis='both',
        which='major',
        labelsize=prop('fontsize_axis_ticks')
    )
    
    axis.spines['top'].set_visible(False)
    axis.spines['right'].set_visible(False)
        
    # build custom legend
    custom_lines = [
        lines.Line2D(
            [0], [0],
            color=rgbcodes['CLR'],
            markersize=prop('legend_marker_size'),
            marker=prop('CLR_marker'),
            ls='None',
            label='CLR (HET)'
        ),
        lines.Line2D(
            [0], [0],
            color=rgbcodes['HiFi'],
            markersize=prop('legend_marker_size'),
            marker=prop('HiFi_marker'),
            ls='None',
            label='HiFi (HET)'
        ),
        lines.Line2D(
            [0], [0],
            color=rgbcodes['CLR'],
            markersize=prop('legend_marker_size'),
            marker=prop('CLR_marker'),
            ls='None',
            label='CLR (HOM)',
            alpha=0.33
        ),
        lines.Line2D(
            [0], [0],
            color=rgbcodes['HiFi'],
            markersize=prop('legend_marker_size'),
            marker=prop('HiFi_marker'),
            ls='None',
            label='HiFi (HOM)',
            alpha=0.33
        )   
    ]
    
    axis.legend(
        handles=custom_lines,
        loc='upper right',
        prop={'size': prop('fontsize_legend')}
    )

    if missing:
        plot_aux.add_incomplete_stamp(axis, 0.5, 0.8)
    
    extra_artists = []

    return fig, extra_artists


if save_plots:
    fig, exart = plot_kmer_stats(df)
    if color_version == 'super_pop':
        out_path += '_spop'
    if color_version == 'red_black':
        out_path += '_rb'
    fig.savefig(out_path + '.png', dpi=600, bbox_inches='tight', extra_artists=exart)
    fig.savefig(out_path + '_lowres.png', dpi=150, bbox_inches='tight', extra_artists=exart)
    fig.savefig(out_path + '.svg', bbox_inches='tight', extra_artists=exart)
        
if print_stats:
    xy_pairs = [
        ('Hap_Illumina', 'Hap_GRCh38', 0.33),
        ('Hap1_Illumina', 'Hap1_GRCh38', 1),
        ('Hap2_Illumina', 'Hap2_GRCh38', 1)
    ]
    for platform in plot_aux.get_sequencing_platforms():
        print('========')
        print(platform)
        subset = df.xs(platform, level='platform', drop_level=False)
        hom_kmers_nonref = subset['Hap_Illumina']
        het1_kmers_nonref = subset['Hap1_Illumina']
        het2_kmers_nonref = subset['Hap2_Illumina']
        het_kmers_nonref = pd.concat([het1_kmers_nonref, het2_kmers_nonref], axis=0, ignore_index=True)
        print('HOM ', hom_kmers_nonref.describe())
        print('HET1 ', het1_kmers_nonref.describe())
        print('HET2 ', het2_kmers_nonref.describe())
        print('HET-both ', het_kmers_nonref.describe())
        

CLR
HOM  count    3.000000e+01
mean     4.789272e+07
std      1.999939e+06
min      4.405159e+07
25%      4.647926e+07
50%      4.764900e+07
75%      4.985939e+07
max      5.128784e+07
Name: Hap_Illumina, dtype: float64
HET1  count    3.000000e+01
mean     3.723569e+07
std      5.312619e+06
min      3.177323e+07
25%      3.407156e+07
50%      3.501127e+07
75%      4.257464e+07
max      4.675949e+07
Name: Hap1_Illumina, dtype: float64
HET2  count    3.000000e+01
mean     3.722236e+07
std      5.212569e+06
min      3.132047e+07
25%      3.400802e+07
50%      3.532438e+07
75%      4.265446e+07
max      4.629424e+07
Name: Hap2_Illumina, dtype: float64
HET-both  count    6.000000e+01
mean     3.722902e+07
std      5.218045e+06
min      3.132047e+07
25%      3.402197e+07
50%      3.521803e+07
75%      4.430946e+07
max      4.675949e+07
dtype: float64
HiFi
HOM  count    1.400000e+01
mean     5.458479e+07
std      1.643337e+06
min      5.200922e+07
25%      5.360801e+07
50%      5.448997e+07
7